In [111]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

df_train = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')


In [112]:
df_train['dteday'] = pd.to_datetime(df_train['dteday'])

if 'total_rentals' not in df_train.columns:
    df_train['total_rentals'] = df_train['casual'] + df_train['registered']

features = ['hr', 'temp_c', 'feels_like_c', 'hum', 'windspeed',
            'weathersit', 'season', 'holiday', 'workingday']
X = df_train[features]
y = df_train['total_rentals']

In [113]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [114]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 119952.8438 - mae: 248.8902 - val_loss: 75974.6016 - val_mae: 200.8599
Epoch 2/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 75773.9219 - mae: 199.1148 - val_loss: 69712.9688 - val_mae: 190.9967
Epoch 3/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 66321.4453 - mae: 181.2816 - val_loss: 58047.7695 - val_mae: 165.5894
Epoch 4/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 57088.3164 - mae: 164.9005 - val_loss: 53183.8711 - val_mae: 158.9070
Epoch 5/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 51075.0273 - mae: 152.3264 - val_loss: 49487.5273 - val_mae: 148.5623
Epoch 6/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 49515.3125 - mae: 146.2532 - val_loss: 47110.6094 - val_mae: 140.5990
Epoch 7/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 47250.0000 - mae: 141.2040 - val_loss: 45455.6094 - val_mae: 138.4104
Epoch 8/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 45417

In [115]:
df_december = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes_december.csv')
df_december['dteday'] = pd.to_datetime(df_december['dteday'])

X_december = df_december[features]
X_december_scaled = scaler.transform(X_december)

In [116]:
predictions = model.predict(X_december_scaled)
df_december['predicted_total'] = predictions.astype(int)

daily_predictions = df_december.groupby('dteday')['predicted_total'].sum().reset_index()
daily_predictions.columns = ['date', 'predicted_total_rentals']

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [117]:
daily_predictions.to_csv('daily_bike_predictions.csv', index=False)

print(f"\nPredictions saved to daily_bike_predictions.csv")
print(f"Predicted {len(daily_predictions)} days in December")
print(daily_predictions.head(10))


Predictions saved to daily_bike_predictions.csv
Predicted 61 days in December
        date  predicted_total_rentals
0 2023-11-01                     6927
1 2023-11-02                     7473
2 2023-11-03                     8407
3 2023-11-04                     9346
4 2023-11-05                    10443
5 2023-11-06                     9206
6 2023-11-07                    11112
7 2023-11-08                     9402
8 2023-11-09                     9452
9 2023-11-10                     3942
